In [ ]:
%pip install ruprompts

In [ ]:
!wget https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/train.tsv
!wget https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/dev.tsv

In [ ]:
import pandas as pd

df = pd.read_csv("train.tsv", sep="\t")
df.drop(["index"], axis=1, inplace=True)
df.to_csv("train.tsv", index=False, sep="\t")

In [ ]:
from ruprompts import Prompt
from transformers import pipeline, GPT2LMHeadModel, AutoTokenizer

backbone_id = "ai-forever/rugpt3large_based_on_gpt2"

prompt = Prompt.from_pretrained("konodyuk/prompt_rugpt3large_detox_russe")
model = GPT2LMHeadModel.from_pretrained(backbone_id)
tokenizer = AutoTokenizer.from_pretrained(backbone_id)

ppln = pipeline("text2text-generation-with-prompt", prompt=prompt, model=model, tokenizer=tokenizer, device=0)

In [ ]:
from datasets import load_dataset

dataset_dict = load_dataset("csv", data_files={"train": "train.tsv", "validation": "dev.tsv"}, sep="\t")
valid_dataset = dataset_dict["validation"]

In [ ]:
from tqdm import tqdm
import transformers

transformers.logging.set_verbosity_error()

beam_count = 10

predictions = []

for i in tqdm(valid_dataset["toxic_comment"]):
    options = ppln(
        i,
        do_sample=False,
        num_beams=beam_count,
        num_return_sequences=beam_count,
    )

    options = [i["generated_text"].replace("<pad>", "") for i in options]
    answer = sorted(options, key=len)[-1]  # get longest answer
    predictions.append(answer)

with open("subm.txt", "w") as f:
    f.writelines(list(map(lambda x: x.replace("\n", " ") + "\n", predictions)))
